In [4]:
import random

def fitness(path, distance_matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += distance_matrix[path[i]][path[i + 1]]
    return cost

def random_route(num_locations):
    route = list(range(1, num_locations))
    random.shuffle(route)
    return [0] + route + [0]

def crossover(parent1, parent2):
    split = random.randint(1, len(parent1) - 2)
    child = parent1[:split] + [node for node in parent2 if node not in parent1[:split]]
    return child

def mutate(route):
    if random.random() < MUTATION_RATE:
        index1 = random.randint(1, len(route) - 2)
        index2 = random.randint(1, len(route) - 2)
        route[index1], route[index2] = route[index2], route[index1]
    return route

def initialize_population(num_locations):
    return [random_route(num_locations) for _ in range(POPULATION_SIZE)]

def genetic_algorithm(distance_matrix, num_locations):
    population = initialize_population(num_locations)
    
    for generation in range(GENERATIONS):
        population = sorted(population, key=lambda x: fitness(x, distance_matrix))
        print(f"Generation {generation}: Best route {population[0]} with cost {fitness(population[0], distance_matrix)}")
        
        top_half = population[:len(population) // 2]
        new_population = top_half[:]
        
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.sample(top_half, 2)
            child = crossover(parent1, parent2)
            new_population.append(mutate(child))
        
        population = new_population

    best_route = min(population, key=lambda x: fitness(x, distance_matrix))
    return best_route, fitness(best_route, distance_matrix)

POPULATION_SIZE = 10
GENERATIONS = 20
MUTATION_RATE = 0.3

num_locations = int(input("Enter the number of locations (including depot): "))
distance_matrix = []

print("Enter the distance matrix:")
for i in range(num_locations):
    row = list(map(int, input(f"Enter distances from location {i}: ").split()))
    distance_matrix.append(row)

start_location = 0
best_route, best_cost = genetic_algorithm(distance_matrix, num_locations)
print(f"\nBest route found: {best_route} with cost: {best_cost}")


Enter the number of locations (including depot): 5
Enter the distance matrix:
Enter distances from location 0: 0 10 15 20 25
Enter distances from location 1: 10 0 35 25 30
Enter distances from location 2: 15 35 0 30 5
Enter distances from location 3: 20 25 30 0 15
Enter distances from location 4: 25 30 5 15 0
Generation 0: Best route [0, 3, 1, 4, 2, 0] with cost 95
Generation 1: Best route [0, 3, 4, 2, 1] with cost 75
Generation 2: Best route [0, 1, 3, 4, 2] with cost 55
Generation 3: Best route [0, 1, 3, 4, 2] with cost 55
Generation 4: Best route [0, 1, 3, 4, 2] with cost 55
Generation 5: Best route [0, 1, 3, 4, 2] with cost 55
Generation 6: Best route [0, 1, 3, 4, 2] with cost 55
Generation 7: Best route [0, 1, 3, 4, 2] with cost 55
Generation 8: Best route [0, 1, 3, 4, 2] with cost 55
Generation 9: Best route [0, 1, 3, 4, 2] with cost 55
Generation 10: Best route [0, 1, 3, 4, 2] with cost 55
Generation 11: Best route [0, 1, 3, 4, 2] with cost 55
Generation 12: Best route [0, 1, 3, 

In [2]:
import random

# Define the fitness function for power grid optimization
def fitness_function(positions, loads, cost_coefficients, loss_factor=0.1):
    """
    Fitness function to evaluate power grid optimization.

    :param positions: List of generator outputs (particle positions).
    :param loads: List of energy demands at each zone.
    :param cost_coefficients: Coefficients for generator cost [a, b, c] for quadratic cost.
    :param loss_factor: Proportional factor for energy loss.
    :return: Total cost (fitness value).
    """
    total_power = sum(positions)
    total_load = sum(loads)
    
    loss = loss_factor * (total_power - total_load) ** 2

    balance_penalty = abs(total_power - total_load) * 100

    generation_cost = sum(
        cost_coefficients[i][0] * positions[i]**2 + cost_coefficients[i][1] * positions[i] + cost_coefficients[i][2]
        for i in range(len(positions))
    )

    return generation_cost + balance_penalty + loss


# PSO Algorithm for Power Grid Optimization
def pso(num_particles, num_dimensions, bounds, max_iterations, loads, cost_coefficients):
    # PSO Parameters
    w_max = 0.9  # Initial inertia weight
    w_min = 0.4  # Final inertia weight
    c1 = 1.5  # Cognitive coefficient
    c2 = 1.5  # Social coefficient
    
    max_velocity = (bounds[1] - bounds[0]) / 2  

    particles = [{'position': [random.uniform(bounds[0], bounds[1]) for _ in range(num_dimensions)],
                  'velocity': [random.uniform(-1, 1) for _ in range(num_dimensions)],
                  'best_position': None,
                  'best_fitness': float('inf')} for _ in range(num_particles)]

    global_best_position = None
    global_best_fitness = float('inf')

    for iteration in range(max_iterations):
        w = w_max - (w_max - w_min) * (iteration / max_iterations)

        for particle in particles:
            current_fitness = fitness_function(particle['position'], loads, cost_coefficients)

            if current_fitness < particle['best_fitness']:
                particle['best_fitness'] = current_fitness
                particle['best_position'] = particle['position'][:]

            if current_fitness < global_best_fitness:
                global_best_fitness = current_fitness
                global_best_position = particle['position'][:]
        
        for particle in particles:
            for d in range(num_dimensions):
                r1 = random.random()  
                r2 = random.random()  
                
                cognitive_velocity = c1 * r1 * (particle['best_position'][d] - particle['position'][d])
                social_velocity = c2 * r2 * (global_best_position[d] - particle['position'][d])
                particle['velocity'][d] = w * particle['velocity'][d] + cognitive_velocity + social_velocity

                particle['velocity'][d] = max(min(particle['velocity'][d], max_velocity), -max_velocity)

                particle['position'][d] += particle['velocity'][d]

                if particle['position'][d] < bounds[0]:
                    particle['position'][d] = bounds[0]
                elif particle['position'][d] > bounds[1]:
                    particle['position'][d] = bounds[1]

    return global_best_position, global_best_fitness


if __name__ == "__main__":
    num_generators = int(input("Enter the number of generators: "))
    loads = list(map(float, input("Enter the energy demands (space-separated): ").split()))
    bounds = [float(input("Enter the lower bound of generator output: ")),
              float(input("Enter the upper bound of generator output: "))]
    max_iterations = int(input("Enter the maximum number of iterations: "))
    num_particles = int(input("Enter the number of particles: "))

    cost_coefficients = []
    for i in range(num_generators):
        print(f"Enter cost coefficients (a, b, c) for Generator {i + 1}:")
        cost_coefficients.append(list(map(float, input().split())))

    best_position, best_fitness = pso(num_particles, num_generators, bounds, max_iterations, loads, cost_coefficients)
    
    print("Optimal Generator Outputs:", best_position)
    print("Minimum Total Cost:", best_fitness)


Enter the number of generators: 3
Enter the energy demands (space-separated): 100 120 150
Enter the lower bound of generator output: 50
Enter the upper bound of generator output: 200
Enter the maximum number of iterations: 100
Enter the number of particles: 30
Enter cost coefficients (a, b, c) for Generator 1:
0.01 2 10
Enter cost coefficients (a, b, c) for Generator 2:
0.02 1.5 5
Enter cost coefficients (a, b, c) for Generator 3:
0.015 1.8 8
Optimal Generator Outputs: [156.09541559174176, 89.5129828440415, 124.3915210859806]
Minimum Total Cost: 1329.381118640167


In [2]:
import numpy as np

def euclidean_distance(city1, city2):
    return np.linalg.norm(city1 - city2)

def create_distance_matrix(cities):
    num_cities = len(cities)
    distance_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                distance_matrix[i][j] = euclidean_distance(cities[i], cities[j])
    return distance_matrix

def ant_colony_optimization(cities, num_ants, alpha, beta, rho, pheromone_init, iterations):
    num_cities = len(cities)
    distance_matrix = create_distance_matrix(cities)
    pheromones = np.full((num_cities, num_cities), pheromone_init)
    best_route = None
    best_distance = float('inf')

    def calculate_transition_probabilities(current_city, visited):
        probabilities = []
        for next_city in range(num_cities):
            if next_city not in visited:
                pheromone = pheromones[current_city][next_city] ** alpha
                heuristic = (1 / distance_matrix[current_city][next_city]) ** beta
                probabilities.append(pheromone * heuristic)
            else:
                probabilities.append(0)
        probabilities = np.array(probabilities)
        return probabilities / probabilities.sum()

    for _ in range(iterations):
        all_routes = []
        all_distances = []

        for ant in range(num_ants):
            visited = []
            current_city = np.random.randint(0, num_cities)
            visited.append(current_city)

            for _ in range(num_cities - 1):
                probabilities = calculate_transition_probabilities(current_city, visited)
                next_city = np.random.choice(range(num_cities), p=probabilities)
                visited.append(next_city)
                current_city = next_city

            visited.append(visited[0])
            all_routes.append(visited)
            total_distance = sum(distance_matrix[visited[i]][visited[i + 1]] for i in range(num_cities))
            all_distances.append(total_distance)

            if total_distance < best_distance:
                best_distance = total_distance
                best_route = visited

        pheromones *= (1 - rho)
        for route, distance in zip(all_routes, all_distances):
            for i in range(num_cities):
                pheromones[route[i]][route[i + 1]] += 1 / distance

    return best_route, best_distance

num_cities = int(input("Enter the number of cities: "))
cities = []
for i in range(num_cities):
    x, y = map(float, input(f"Enter coordinates of city {i + 1} (x y): ").split())
    cities.append(np.array([x, y]))

num_ants = int(input("Enter the number of ants: "))
alpha = float(input("Enter the importance of pheromone (alpha): "))
beta = float(input("Enter the importance of heuristic information (beta): "))
rho = float(input("Enter the evaporation rate (rho): "))
pheromone_init = float(input("Enter the initial pheromone value: "))
iterations = int(input("Enter the number of iterations: "))

best_route, best_distance = ant_colony_optimization(cities, num_ants, alpha, beta, rho, pheromone_init, iterations)

print("\nBest Route (Order of Cities):", best_route)
print("Best Distance:", best_distance)


Enter the number of cities: 5
Enter coordinates of city 1 (x y): 0 0 
Enter coordinates of city 2 (x y): 2 3
Enter coordinates of city 3 (x y): 5 1 
Enter coordinates of city 4 (x y): 6 4
Enter coordinates of city 5 (x y): 8 0
Enter the number of ants: 10
Enter the importance of pheromone (alpha): 1
Enter the importance of heuristic information (beta): 2
Enter the evaporation rate (rho): 0.2
Enter the initial pheromone value: 1.5
Enter the number of iterations: 100

Best Route (Order of Cities): [0, 1, 3, 4, 2, 0]
Best Distance: 20.462090029842393


In [1]:
import numpy as np

def fitness_function(path, obstacles, target):
    path_length = np.sum(np.linalg.norm(np.diff(path, axis=0), axis=1))
    collision_penalty = 0

    for point in path:
        for obs in obstacles:
            obs_x, obs_y, radius = obs
            distance_to_obs = np.linalg.norm(point - np.array([obs_x, obs_y]))
            if distance_to_obs < radius:
                collision_penalty += 1e6

    distance_to_target = np.linalg.norm(path[-1] - target)
    return path_length + collision_penalty + distance_to_target

def levy_flight(dim):
    beta = 1.5
    u = np.random.normal(0, 1, dim)
    v = np.random.normal(0, 1, dim)
    step = u / (np.abs(v) ** (1 / beta))
    return step

def cuckoo_search_robot(num_nests, max_iter, waypoints, lower_bound, upper_bound, start, target, obstacles):
    dim = waypoints * 2
    nests = np.random.uniform(lower_bound, upper_bound, (num_nests, dim))
    nests = nests.reshape((num_nests, waypoints, 2))
    fitness = np.array([fitness_function(np.vstack([start, nest, target]), obstacles, target) for nest in nests])

    best_nest = nests[np.argmin(fitness)]
    best_fitness = min(fitness)

    for _ in range(max_iter):
        for i in range(num_nests):
            new_nest = nests[i] + levy_flight(dim).reshape(waypoints, 2)
            new_nest = np.clip(new_nest, lower_bound, upper_bound)
            new_fitness = fitness_function(np.vstack([start, new_nest, target]), obstacles, target)
            
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        current_best_idx = np.argmin(fitness)
        current_best_fitness = fitness[current_best_idx]
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_nest = nests[current_best_idx]

        abandon_prob = 0.25
        worst_nests_idx = np.argsort(fitness)[-int(abandon_prob * num_nests):]
        for idx in worst_nests_idx:
            nests[idx] = np.random.uniform(lower_bound, upper_bound, (waypoints, 2))
            fitness[idx] = fitness_function(np.vstack([start, nests[idx], target]), obstacles, target)

    return best_nest, best_fitness

num_nests = int(input("Enter the number of nests: "))
max_iter = int(input("Enter the number of iterations: "))
waypoints = int(input("Enter the number of waypoints: "))
lower_bound = float(input("Enter the lower bound of the search space: "))
upper_bound = float(input("Enter the upper bound of the search space: "))
start_x, start_y = map(float, input("Enter the start position (x y): ").split())
target_x, target_y = map(float, input("Enter the target position (x y): ").split())
num_obstacles = int(input("Enter the number of obstacles: "))
obstacles = []

for i in range(num_obstacles):
    obs_x, obs_y, radius = map(float, input(f"Enter obstacle {i+1} (x y radius): ").split())
    obstacles.append((obs_x, obs_y, radius))

start = np.array([start_x, start_y])
target = np.array([target_x, target_y])

best_path, best_value = cuckoo_search_robot(num_nests, max_iter, waypoints, lower_bound, upper_bound, start, target, obstacles)

print("\nBest Path (Waypoints):")
print(np.vstack([start, best_path, target]))
print("Best Fitness Value:", best_value)


Enter the number of nests: 40
Enter the number of iterations: 200
Enter the number of waypoints: 6
Enter the lower bound of the search space: 0
Enter the upper bound of the search space: 10
Enter the start position (x y): 0 0
Enter the target position (x y): 10 10
Enter the number of obstacles: 3
Enter obstacle 1 (x y radius): 3 3 1
Enter obstacle 2 (x y radius): 5 1.5 5
Enter obstacle 3 (x y radius): 8 8 3

Best Path (Waypoints):
[[ 0.          0.        ]
 [ 0.          0.76102858]
 [ 0.89222365  7.86675543]
 [ 1.23524835  9.16650639]
 [ 2.3496534   9.07199489]
 [ 3.02386686  9.55571965]
 [ 5.34667346  9.79552325]
 [10.         10.        ]]
Best Fitness Value: 1000018.2079718587
